### 00. 데이터 불러오기

In [1]:
import pandas as pd
dataset = pd.read_csv('./total_price.csv')

In [2]:
del dataset['Unnamed: 0']

In [3]:
dataset['l1'] = dataset['l1'].apply(lambda x : str(x)).apply(lambda x : x[1:]).apply(lambda x : float(x))
dataset['l2'] = dataset['l2'].apply(lambda x : str(x)).apply(lambda x : x[1:]).apply(lambda x : float(x))
dataset['l3'] = dataset['l3'].apply(lambda x : str(x)).apply(lambda x : x[1:]).apply(lambda x : float(x))
dataset['l4'] = dataset['l4'].apply(lambda x : str(x)).apply(lambda x : x[1:]).apply(lambda x : float(x))

### 01. 예측하고 싶은 종목

In [4]:
code = 239340

### 02. 예측하고 싶은 종목에 대하여 모델 생성 및 예측

In [5]:
import warnings
warnings.filterwarnings(action='ignore') 

In [6]:
data = dataset[dataset['code'] == code]
data['target'] = data['close'].shift(-1)
data = data.dropna(axis=0)
X = data[data.columns[1:-2]]
y = data[data.columns[-1]]

In [7]:
data

,date,open,high,low,close,trading_volume,score,index,probability,l1,l2,l3,l4,lgap,lrate,code,target
1901411,20160622,1985,2000,1975,2000,53805,1.0,1.0,30.000,2012.5,2000.0,1987.5,1975.0,37.5,2,239340,1980.0
1901412,20160623,2000,2000,1980,1980,12076,1.0,1.0,30.000,2000.0,1990.0,1980.0,1970.0,30.0,2,239340,1975.0
1901413,20160624,1985,1985,1975,1975,14630,1.0,1.0,30.000,1985.0,1980.0,1975.0,1970.0,15.0,1,239340,1980.0
1901414,20160627,1990,1990,1975,1980,63,1.0,1.0,30.000,1992.5,1985.0,1977.5,1970.0,22.5,1,239340,1980.0
1901415,20160628,1980,1980,1980,1980,2780,1.0,1.0,30.000,1980.0,1980.0,1980.0,1980.0,0.0,0,239340,1985.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1902307,20200218,4560,4635,4470,4490,192748,8725.0,73.0,79.981,4645.0,4562.5,4480.0,4397.5,247.5,6,239340,4500.0
1902308,20200219,4470,4645,4325,4500,400766,8735.0,74.0,79.982,4732.5,4572.5,4412.5,4252.5,480.0,11,239340,4425.0
1902309,20200220,4500,4540,4300,4425,202980,9340.0,100.0,79.990,4602.5,4482.5,4362.5,4242.5,360.0,8,239340,4265.0
1902310,20200221,4335,4655,4265,4265,484867,8725.0,73.0,79.981,4655.0,4460.0,4265.0,4070.0,585.0,14,239340,4275.0


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.10, random_state=1234)

In [9]:
import lightgbm as lgb
train_ds = lgb.Dataset(X_train, label = y_train) 
valid_ds = lgb.Dataset(X_valid, label = y_valid) 

In [10]:
params = {'learning_rate': 0.01, 
          'max_depth': 5, 
          'boosting': 'gbdt', 
          'objective': 'regression', 
          'metric': 'mse', 
          'is_training_metric': True, 
          'num_leaves': 144, 
          'feature_fraction': 0.9, 
          'bagging_fraction': 0.7, 
          'bagging_freq': 5, 
          'seed':1234}

In [11]:
model = lgb.train(params, train_ds, 1000, valid_ds, verbose_eval=100, early_stopping_rounds=100)

Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 329033
[200]	valid_0's l2: 45436
[300]	valid_0's l2: 12815.4
[400]	valid_0's l2: 10235.2
Early stopping, best iteration is:
[382]	valid_0's l2: 9968.99


In [12]:
predict_train = model.predict(X_train)
predict_test = model.predict(X_valid)

In [13]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_valid, predict_test)
r2 = r2_score(y_valid, predict_test)

In [14]:
print('Mean squared error: ', mse)
print('Root Mean squared error: ', mse**.5)
print('R2 score: ', r2)

Mean squared error:  9968.986224832855
Root Mean squared error:  99.84481070557877
R2 score:  0.996102396326303


In [15]:
import numpy as np
aaa = 100 * (np.array(y_valid) - np.array(X_valid.close)) / np.array(X_valid.close)
bbb = 100 * (model.predict(X_valid) - np.array(X_valid.close)) / np.array(X_valid.close)

In [16]:
predict = pd.DataFrame({'오늘 실제 종가' : np.array(X_valid.close),
                        '내일 실제 종가' : np.array(y_valid), 
                        '내일 예측 종가' : model.predict(X_valid),
                        '실제 내일 변동(%)' : aaa,
                        '모델 예상 내일 변동(%)' : bbb})
round(predict, 2)

,오늘 실제 종가,내일 실제 종가,내일 예측 종가,실제 내일 변동(%),모델 예상 내일 변동(%)
0,2065,2055.0,2079.41,-0.48,0.70
1,2085,2085.0,2103.06,0.00,0.87
2,1985,1985.0,2003.54,0.00,0.93
3,2085,2085.0,2103.06,0.00,0.87
4,2030,2035.0,2051.75,0.25,1.07
...,...,...,...,...,...
86,2020,2020.0,2048.92,0.00,1.43
87,2085,2085.0,2103.06,0.00,0.87
88,2010,2015.0,2020.51,0.25,0.52
89,2045,2040.0,2063.51,-0.24,0.90


### 03. Test  데이터 예시

In [17]:
# https://lab.donutz.co/krx/products/239340?sdate=200226&edate=200305

In [18]:
ttt = [{"date":"2020-02-26","open":4215,"high":4430,"low":4215,"close":4430,"trading_volume":166858,"score":8740,"index":75,"probability":79.9822253821543,"l1":4537.5,"l2":4430,"l3":4322.5,"l4":4215,"lgap":322.5,"lrate":7},{"date":"2020-02-27","open":4430,"high":4490,"low":4190,"close":4280,"trading_volume":250062,"score":9325,"index":98,"probability":79.9895887558563,"l1":4535,"l2":4385,"l3":4235,"l4":4085,"lgap":450,"lrate":11},{"date":"2020-02-28","open":4200,"high":4245,"low":3960,"close":4020,"trading_volume":302100,"score":8890,"index":72,"probability":79.9807135969142,"l1":4275,"l2":4132.5,"l3":3990,"l4":3847.5,"lgap":427.5,"lrate":11},{"date":"2020-03-02","open":3980,"high":4120,"low":3970,"close":4110,"trading_volume":188277,"score":8880,"index":72,"probability":79.9807135969142,"l1":4190,"l2":4115,"l3":4040,"l4":3965,"lgap":225,"lrate":5},{"date":"2020-03-03","open":4220,"high":4220,"low":4070,"close":4070,"trading_volume":150586,"score":8880,"index":72,"probability":79.9807135969142,"l1":4220,"l2":4145,"l3":4070,"l4":3995,"lgap":225,"lrate":6},{"date":"2020-03-04","open":4050,"high":4255,"low":4050,"close":4185,"trading_volume":217026,"score":9325,"index":98,"probability":79.9895887558563,"l1":4322.5,"l2":4220,"l3":4117.5,"l4":4015,"lgap":307.5,"lrate":7},{"date":"2020-03-05","open":4260,"high":4275,"low":4200,"close":4235,"trading_volume":77739,"score":9320,"index":98,"probability":79.9895887558563,"l1":4292.5,"l2":4255,"l3":4217.5,"l4":4180,"lgap":112.5,"lrate":3}]

In [19]:
test = pd.DataFrame(ttt)

In [20]:
X_test = test[test.columns[1:]]

In [21]:
X_test

,open,high,low,close,trading_volume,score,index,probability,l1,l2,l3,l4,lgap,lrate
0,4215,4430,4215,4430,166858,8740,75,79.982225,4537.5,4430.0,4322.5,4215.0,322.5,7
1,4430,4490,4190,4280,250062,9325,98,79.989589,4535.0,4385.0,4235.0,4085.0,450.0,11
2,4200,4245,3960,4020,302100,8890,72,79.980714,4275.0,4132.5,3990.0,3847.5,427.5,11
3,3980,4120,3970,4110,188277,8880,72,79.980714,4190.0,4115.0,4040.0,3965.0,225.0,5
4,4220,4220,4070,4070,150586,8880,72,79.980714,4220.0,4145.0,4070.0,3995.0,225.0,6
5,4050,4255,4050,4185,217026,9325,98,79.989589,4322.5,4220.0,4117.5,4015.0,307.5,7
6,4260,4275,4200,4235,77739,9320,98,79.989589,4292.5,4255.0,4217.5,4180.0,112.5,3


In [22]:
model.predict(X_test)

array([4438.11469086, 4383.97125345, 4258.87259201, 4013.37017953,
       4012.64159223, 4272.695814  , 4408.05823972])

In [23]:
ccc = 100 * (np.array(test[test.columns[1:]].close.shift(-1)) - np.array(test[test.columns[1:]].close))\
                      /np.array(test[test.columns[1:]].close)
ddd = 100 * (model.predict(test[test.columns[1:]]) - np.array(test[test.columns[1:]].close))\
                      /np.array(test[test.columns[1:]].close)

In [24]:
predict = pd.DataFrame({'오늘 실제 종가' : np.array(test[test.columns[1:]].close),
                        '내일 실제 종가' : np.array(test[test.columns[1:]].close.shift(-1)),
                        '내일 예측 종가' : model.predict(test[test.columns[1:]]),
                        '실제 내일 변동(%)' : ccc,
                        '모델 예상 내일 변동(%)' : ddd})
round(predict, 2)

,오늘 실제 종가,내일 실제 종가,내일 예측 종가,실제 내일 변동(%),모델 예상 내일 변동(%)
0,4430,4280.0,4438.11,-3.39,0.18
1,4280,4020.0,4383.97,-6.07,2.43
2,4020,4110.0,4258.87,2.24,5.94
3,4110,4070.0,4013.37,-0.97,-2.35
4,4070,4185.0,4012.64,2.83,-1.41
5,4185,4235.0,4272.70,1.19,2.10
6,4235,NaN,4408.06,NaN,4.09
